# Import packages

In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, sys
from pathlib import Path
for p in [Path.cwd()] + list(Path.cwd().parents):
    if p.name == 'Multifirefly-Project':
        os.chdir(p)
        sys.path.insert(0, str(p / 'multiff_analysis/multiff_code/methods'))
        break

from data_wrangling import specific_utils, process_monkey_information, general_utils
from pattern_discovery import pattern_by_trials, pattern_by_trials, cluster_analysis, organize_patterns_and_features
from visualization.matplotlib_tools import plot_behaviors_utils
from neural_data_analysis.neural_analysis_tools.get_neural_data import neural_data_processing
from neural_data_analysis.neural_analysis_tools.visualize_neural_data import plot_neural_data, plot_modeling_result
from neural_data_analysis.neural_analysis_tools.model_neural_data import transform_vars, neural_data_modeling, drop_high_corr_vars, drop_high_vif_vars
from neural_data_analysis.topic_based_neural_analysis.neural_vs_behavioral import prep_monkey_data, prep_target_data, neural_vs_behavioral_class
from neural_data_analysis.topic_based_neural_analysis.planning_and_neural import planning_and_neural_class, pn_utils, pn_helper_class
from neural_data_analysis.neural_analysis_tools.cca_methods import cca_class
from neural_data_analysis.neural_analysis_tools.cca_methods import cca_class, cca_utils, cca_cv_utils
from neural_data_analysis.neural_analysis_tools.cca_methods.cca_plotting import cca_plotting, cca_plot_lag_vs_no_lag, cca_plot_cv
from machine_learning.ml_methods import regression_utils, ml_methods_utils, regz_regression_utils, ml_methods_class, classification_utils, ml_plotting_utils
from neural_data_analysis.neural_analysis_tools.glm_tools import glm_class
from neural_data_analysis.topic_based_neural_analysis.planning_and_neural import planning_and_neural_class, pn_utils, pn_helper_class, pn_aligned_by_seg, pn_aligned_by_event
from neural_data_analysis.neural_analysis_tools.glm_tools.tpg import glm_bases, plot_glm_fit, glm_on_multiff
import sys
import math
import gc
import subprocess
from pathlib import Path
from importlib import reload

# Third-party imports
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc
from scipy import linalg, interpolate
from scipy.signal import fftconvolve
from scipy.io import loadmat
from scipy import sparse
from numpy import pi

# Machine Learning imports
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.multivariate.cancorr import CanCorr

# Neuroscience specific imports
import neo
import rcca

plt.rcParams["animation.html"] = "html5"
os.environ['KMP_DUPLICATE_LIB_OK']='True'
rc('animation', html='jshtml')
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
matplotlib.rcParams['animation.embed_limit'] = 2**128
pd.set_option('display.float_format', lambda x: '%.5f' % x)
np.set_printoptions(suppress=True)
print("done")

%load_ext autoreload
%autoreload 2

# Retrieve data

## get data

In [ ]:
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0330"

In [ ]:
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Schro/data_0416"

In [ ]:
reload(pn_helper_class)

In [ ]:
reduce_y_var_lags = False
planning_data_by_point_exists_ok = True
y_data_exists_ok = True

pn = pn_aligned_by_seg.PlanningAndNeuralSegmentAligned(raw_data_folder_path=raw_data_folder_path)
pn.prep_data_to_analyze_planning(planning_data_by_point_exists_ok=planning_data_by_point_exists_ok)
pn.planning_data_by_point, cols_to_drop = general_utils.drop_columns_with_many_nans(
    pn.planning_data_by_point)
pn.get_x_and_y_data_for_modeling(exists_ok=y_data_exists_ok, reduce_y_var_lags=reduce_y_var_lags)
pn.rebin_data_in_new_segments(segment_duration=2, rebinned_max_x_lag_number=2)

for col in ['cur_ff_visible_dummy', 'nxt_ff_visible_dummy', 'cur_ff_in_memory_dummy', 'nxt_ff_in_memory_dummy']:
    pn.rebinned_y_var[col] = (pn.rebinned_y_var[col] > 0).astype(int)

In [ ]:
stop!

# design_df

In [ ]:
from neural_data_analysis.neural_analysis_tools.model_neural_data import design_bases
data = pn.rebinned_y_var.copy()
# spike_data = pn.rebinned_x_var['cluster_0']
dt = pn.bin_width

design_df, y, meta = design_bases.design_bases_for_behav_data(
    dt=dt, trial_ids=data["new_segment"],
    cur_vis=data["cur_ff_in_memory_dummy"], nxt_vis=data["nxt_ff_in_memory_dummy"],
    cur_dist=data["cur_ff_distance"], nxt_dist=data["nxt_ff_distance"],
    cur_angle=data["cur_ff_angle"], nxt_angle=data["nxt_ff_angle"],
    speed=data["monkey_speed"], curvature=data["curv_of_traj"],
    angular_speed=data["monkey_dw"],
    stop=data['monkey_speeddummy'],
    capture=data['capture_target_dummy'],
)


In [ ]:
pn.rebinned_y_var.columns.tolist()

# CCA

https://medium.com/@pozdrawiamzuzanna/canonical-correlation-analysis-simple-explanation-and-python-example-a5b8e97648d2

## conduct cca

In [ ]:
X1 = pn.rebinned_x_var.drop(columns=['bin', 'segment', 'new_bin', 'new_segment'], errors='ignore').copy()
X2 = pn.rebinned_y_var.copy()
X1_reduced = design_bases.reduce_df(X1)
X2_reduced = design_bases.reduce_df(X2)

X1_lags_reduced = X1_reduced.copy()


In [ ]:
X2_lags = pd.concat([X2, design_df], axis=1)
X2_lags_reduced = design_bases.reduce_df(X2_lags)

In [ ]:
cca_no_lag = cca_class.CCAclass(X1=X1_reduced, X2=X2_reduced, lagging_included=False)
cca_no_lag.conduct_cca()

cca_lags = cca_class.CCAclass(X1=X1_lags_reduced, X2=X2_lags_reduced, lagging_included=True)
cca_lags.conduct_cca()

print(f'X1_reduced.shape: {X1_reduced.shape}')
print(f'X2_reduced.shape: {X2_reduced.shape}')
print(f'X1_lags_reduced.shape: {X1_lags_reduced.shape}')
print(f'X2_lags_reduced.shape: {X2_lags_reduced.shape}')

cca_inst = cca_lags